In [1]:
import argparse
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta
import os
from sklearn.metrics import confusion_matrix
from PIL import Image

import utils

In [2]:
# set global variables
num_classes = 0
dataset_train = None
cls_train = None
labels_train = None

dataset_test = None
cls_test = None
labels_test = None

session = None
x = None
y_true = None
y_pred_cls = None
optimizer = None
accuracy = None
cost=None
global_step = None
x_image=None
cls_pred=None
###############
filter_size1 = 7        # Convolution filters are 7 x 7 pixels.
num_filters1 = 16       # There are 16 of these filters.
filter_size2 = 7        # Convolution filters are 7 x 7 pixels.
num_filters2 = 36       # There are 36 of these filters.

# Fully-connected layer.
fc_size = 1024          # Number of neurons in fully-connected layer.



In [3]:
def random_batch(batch_size):
    global dataset_train, labels_train
    # Number of images (transfer-values) in the training-set.
    num_images = len(dataset_train)
    # Create a random index.
    idx = np.random.choice(num_images,size=batch_size,replace=False)
    # get a set of random transfer values
    x_batch = dataset_train[idx]
    y_batch = labels_train[idx]
    return x_batch, y_batch


In [4]:
def optimize(num_iterations, batch_size = 30):
    global optimizer, accuracy, global_step, x_image
    start_time = time.time()
    for i in range(num_iterations):
        x_batch, y_true_batch = random_batch(batch_size)
        feed_dict_train = {x_image: x_batch, y_true: y_true_batch}
        session.run(optimizer, feed_dict=feed_dict_train)
        # Print accuracy every 100 iterations
        if (i % 50 == 0) or (i == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Global Step: {0:>4}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i+1, batch_acc))

    # Print time
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [5]:
def print_test_accuracy(show_example_errors=False,show_confusion_matrix=False):
    global dataset_test, labels_test, cls_test,y_pred_cls,x_image,cls_pred
    num_test = len(dataset_test)
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0
    test_batch_size = 30
    while i < num_test:
        j = min(i + test_batch_size, num_test)
        images = dataset_test[i:j, :]
        labels = labels_test[i:j,  :]
        feed_dict = {x_image: images,y_true: labels}
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        i = j

    cls_true = cls_test
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))


In [6]:
dataset_train = np.load('shape_train.npy')
dataset_test = np.load('shape_test.npy')
# load class
cls_train = np.load('shape_train_cls.npy')
cls_test = np.load('shape_test_cls.npy')
num_train_samples = len(cls_train)
num_test_samples = len(cls_test)
print cls_train
num_classes = int(np.max(cls_train) + 1)
print "num_classes {}".format(num_classes)

num_classes1 = int(np.max(cls_test) + 1)
print "num_classes {}".format(num_classes1)

# get labels
labels_train = np.zeros((num_train_samples, num_classes))
labels_test = np.zeros((num_test_samples, num_classes))

for i in xrange(num_train_samples):
    labels_train[i, cls_train[i]] = 1

for i in xrange(num_test_samples):
    labels_test[i, cls_test[i]] = 1

input_dim = dataset_train.shape[1]
num_channels = dataset_train.shape[3]
print "input_dim {}".format(input_dim)
print "num_channels {}".format(num_channels)

# create place holder variables

x_image = tf.placeholder(tf.float32 ,shape=[None, input_dim, input_dim, num_channels], name='x_image')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

layer_conv1, weights_conv1 = utils.new_conv_layer(input=x_image, num_input_channels=num_channels, filter_size=filter_size1, num_filters=num_filters1,use_pooling=True)
print layer_conv1

layer_conv2, weights_conv2 = utils.new_conv_layer(input=layer_conv1,num_input_channels=num_filters1,filter_size=filter_size2,num_filters=num_filters2,use_pooling=True)	
print layer_conv2

layer_flat, num_features = utils.flatten_layer(layer_conv2)
print layer_flat

layer_fc1 = utils.new_fc_layer(input=layer_flat,num_inputs=num_features,num_outputs=fc_size,use_relu=True)
print layer_fc1
layer_fc2 = utils.new_fc_layer(input=layer_fc1,num_inputs=fc_size,num_outputs=num_classes,use_relu=False)
print layer_fc2

y_pred = tf.nn.softmax(layer_fc2)

y_pred_cls = tf.argmax(y_pred, dimension=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
#promedio
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

session = tf.Session()

session.run(tf.global_variables_initializer())

train_batch_size = 5

optimize(num_iterations = 1, batch_size = train_batch_size)

print_test_accuracy()

optimize(num_iterations = 100, batch_size = train_batch_size)

print_test_accuracy()

session.close()

print cls_test
print cls_pred
# compute accuracy for each class
cmatrix = confusion_matrix(cls_test, cls_pred, labels=range(num_classes))
cmatrix = np.transpose(cmatrix) # cols trueLabel and rows predicted
print 'Confusion matrix'
for i in range(num_classes):
    accuracy_class = 0
    if np.sum(cmatrix[:,i]) > 0:
        accuracy_class = float(cmatrix[i,i]) / float(np.sum(cmatrix[:,i]))
    print "Accuracy class {} : {}".format(i+1, accuracy_class)


[0 0 0 0 0 0 0 1 1 1 1 1 1 1]
num_classes 2
num_classes 2
input_dim 200
num_channels 3
Tensor("Relu:0", shape=(?, 100, 100, 16), dtype=float32)
Tensor("Relu_1:0", shape=(?, 50, 50, 36), dtype=float32)
Tensor("Reshape:0", shape=(?, 90000), dtype=float32)
Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)
Tensor("add_3:0", shape=(?, 2), dtype=float32)
Global Step:    1, Training Batch Accuracy:  40.0%
Time usage: 0:00:02
Accuracy on Test-Set: 50.0% (3 / 6)
Global Step:    1, Training Batch Accuracy:  20.0%
Global Step:   51, Training Batch Accuracy: 100.0%
Global Step:  100, Training Batch Accuracy: 100.0%
Time usage: 0:04:23
Accuracy on Test-Set: 33.3% (2 / 6)
Global Step:    1, Training Batch Accuracy: 100.0%
Global Step:   51, Training Batch Accuracy: 100.0%
Global Step:  100, Training Batch Accuracy: 100.0%
Time usage: 0:04:28
Accuracy on Test-Set: 33.3% (2 / 6)
[0 0 0 1 1 1]
[1 1 0 1 0 0]
Confusion matrix
Accuracy class 1 : 0.333333333333
Accuracy class 2 : 0.333333333333
